**<center><font size=5>E.T. Signal Search</font></center>**

----
# Background

##### "Are we alone in the Universe?"

It's one of the most profound human questions.
As technology improves, we're finding new and more powerful ways to seek for answers.

The listen teams at Berkeley University, scans signals from millions of stars in the universe using world's most powerful telescopes - but it is not that easy, by the time, humans have built enormous numbers of radio device that can be detected as signals as well and distract our signals interpetation.
In order to handle this challange today, they are using two different methods:
1. They intersperses scans of target stars that apprears in more than one regions of the sky - because it's probably isn't coming from the direction of target star.
2. They discards signals that don't change their frequency - because it means that they are probably nearby the telescope.

These two methods are quite effective, but they can be improved. The process today misses interesting signals. particularly those with complex time or frequency structure, and those in regions of the spectrum with lots of interference

(from kaggle competition - [im a link])

## Our Task

We are about to use unsupervised learning skills to help identify anomalous signals in these scans.

##### But, How?
1. EDA - summarizing our data main characteristics
2. Instantiate GPU/TPU strategy
3. Data preprocessing
4. Build appropriate models
5. Train and test the models
6. Conclusions
